# Initialization

## Inport modules

In [1]:
%matplotlib ipympl
import os, sys
import time
import pyvisa
import warnings
import numpy as np
import qcodes as qc
import plotly.graph_objects as go
import matplotlib.pyplot as plt

def find_qcodes_local_dir():
    dirpath = os.getcwd()
    while True:
        dirpath, folder_name = os.path.split(dirpath)
        if folder_name == 'QCoDeS_local':
            return os.path.join(dirpath, folder_name)
        if not folder_name:  # Reached the root directory
            return None
qcodes_local_dir = find_qcodes_local_dir()
sys.path.append(f'{qcodes_local_dir}')

from tqdm import tqdm
from pprint import pprint
from time import sleep, monotonic, time
from IPython.display import clear_output
from qcodes.dataset.measurements import Measurement
from qcodes.utils.metadata import diff_param_values
from qcodes.dataset.plotting import plot_dataset, plot_by_id
from qcodes import Parameter, ManualParameter, ScaledParameter
from qcodes.instrument.specialized_parameters import ElapsedTimeParameter
from sweeps_v2 import do1d, do2d, time_sweep, measure_until, do1d_until

print('Imported all modules, QCoDeS version:', qc.__version__, 'initialized')

# warnings.filterwarnings('ignore')

Imported all modules, QCoDeS version: 0.38.1 initialized


In [2]:
def find_qcodes_local_dir():
    dirpath = os.getcwd()
    while True:
        dirpath, folder_name = os.path.split(dirpath)
        if folder_name == 'QCoDeS_local':
            return os.path.join(dirpath, folder_name)
        if not folder_name:  # Reached the root directory
            return None
qcodes_local_dir = find_qcodes_local_dir()
qcodes_local_dir

'c:\\Users\\admin\\Documents\\GitHub\\QCoDeS_local'

In [3]:
def find_init_fridge_v2_py():
    for root, dirs, files in os.walk(qcodes_local_dir):
        if 'init_fridge_v2.py' in files:
            return os.path.join(root, 'init_fridge_v2.py')
    return None

init_fridge_v2_py_path = find_init_fridge_v2_py()
init_fridge_v2_py_path

'c:\\Users\\admin\\Documents\\GitHub\\QCoDeS_local\\init_fridge_v2.py'

In [4]:
fridge_name = 'BF1'
%run "{init_fridge_v2_py_path}" {fridge_name}

Found init_BF1_v2.py at: c:\Users\admin\Documents\GitHub\QCoDeS_local\fridge_specific\init_BF1_v2.py
Error connecting to TCPIP0::K-E5080B-02870::inst0::INSTR: VI_ERROR_RSRC_NFOUND (-1073807343): Insufficient location information or the requested device or resource is not present in the system.
Error connecting to ASRL1::INSTR: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.
Error connecting to ASRL3::INSTR: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.
Error connecting to GPIB0::5::INSTR: VI_ERROR_TMO (-1073807339): Timeout expired before operation completed.
Device: GPIB0::7::INSTR
IDN: KEITHLEY INSTRUMENTS,MODEL DMM6500,04518181,1.7.7b

Connected to: KEITHLEY INSTRUMENTS DMM6500 (serial:04518181, firmware:1.7.7b) in 0.02s
Added Keithley DMM6500 at GPIB0::7::INSTR to the station.
Device: GPIB0::23::INSTR
IDN: KEITHLEY INSTRUMENTS INC.,MODEL 2000,0775539,A13  /A02  

Connected to: KEITHLEY INSTRUMENTS INC. 2000 (serial:0775539, firmware

[K2000(Keithley2000)] Snapshot: Could not update parameter: trigger_count


Added Keithley 2000 at GPIB0::23::INSTR to the station.
Initialized IVVI-rack in 0.01s
Added IVVI at ASRL3::INSTR to the station.
Failed to ping 169.254.115.3: [WinError 10060] 連線嘗試失敗，因為連線對象有一段時間並未正確回應，或是連線建立失敗，因為連線的主機無法回應。
Failed to ping 169.254.115.2: [WinError 10060] 連線嘗試失敗，因為連線對象有一段時間並未正確回應，或是連線建立失敗，因為連線的主機無法回應。
Failed to ping 169.254.115.1: [WinError 10060] 連線嘗試失敗，因為連線對象有一段時間並未正確回應，或是連線建立失敗，因為連線的主機無法回應。
One or more pings failed, cannot establish connection to all magnets.

Station list:
{'DMM6500': <Keithley_6500: DMM6500>,
 'K2000': <Keithley2000: K2000>,
 'ivvi': <IVVI: ivvi>}


In [ ]:
snapshot_of_station = station.snapshot()
pprint(snapshot_of_station)

## IVVI sensitive check

In [7]:
ivvi.dac1.step = 100
ivvi.dac1.inter_delay = 0.01
ivvi.dac2.step = 100
ivvi.dac2.inter_delay = 0.01
ivvi.dac3.step = 5
ivvi.dac3.inter_delay = 0.1

In [5]:
ivvi.dac5.step = 10
ivvi.dac5.inter_delay = 0.1
ivvi.dac1(0)
ivvi.dac2(0)
ivvi.dac3(0)
ivvi.dac4(0)
ivvi.dac5(0)

#  Database initialisation

In [6]:
def find_init_database_v2_py():
    for root, dirs, files in os.walk(qcodes_local_dir):
        if 'init_database_v2.py' in files:
            return os.path.join(root, 'init_database_v2.py')
    return None

init_database_v2_py_path = find_init_database_v2_py()
init_database_v2_py_path

'c:\\Users\\admin\\Documents\\GitHub\\QCoDeS_local\\init_database_v2.py'

In [7]:
reuse = True
%run "{init_database_v2_py_path}" {reuse}

Using existing database file:
c:\Users\admin\Documents\GitHub\QCoDeS_local\personal_scripts\Albert\PtTe2\018\018_2025-02-26_01.db


## Defining scaled parameters

### IVVI version

In [8]:
Irange = Parameter('Irange', label='current range', unit='', set_cmd=None, get_cmd=None)
Vrange = Parameter('Vrange', label='voltage range', unit='', set_cmd=None, get_cmd=None)
TGrange = Parameter('TGrange', label='Top gate range', unit='', set_cmd=None, get_cmd=None)
TGrange2 = Parameter('TGrange2', label='Top gate range2', unit='', set_cmd=None, get_cmd=None)

#division paraemtrs required for simplier update: they used as an argument for scaled paraemtrs of sorce instruments

Irange_division = Parameter('Irange_division', 
                            label='scaled current range', unit='', 
                            set_cmd=None, get_cmd=lambda : 1000 / Irange())

Vrange_division = Parameter('Vrange_division', 
                            label='scaled voltage range', unit='', 
                            set_cmd=None, get_cmd=lambda : 1000 / Vrange())

TGrange_division = Parameter('TGrange_division', 
                             label='scaled Top gate range', unit='', 
                             set_cmd=None, get_cmd=lambda : 1000 / TGrange())

TGrange_division2 = Parameter('TGrange_division2', 
                             label='scaled Top gate range2', unit='', 
                             set_cmd=None, get_cmd=lambda : 1000 / TGrange2())

Irange_AC_division = Parameter('Irange_AC_division', 
                               label='scaled current range for lockin', unit='', 
                               set_cmd=None, get_cmd=lambda : 100 / Irange())

Vrange_AC_division = Parameter('Vrange_AC_division', 
                               label='scaled voltage range for lockin', unit='', 
                               set_cmd=None, get_cmd=lambda : 100 / Vrange())

Gain_K1 = Parameter('Gain_K1', label='Gain K1', unit='', set_cmd=None, get_cmd=None)
Gain_K2 = Parameter('Gain_K2', label='Gain K2', unit='', set_cmd=None, get_cmd=None)
Gain_L1 = Parameter('Gain_L1', label='Gain L1', unit='', set_cmd=None, get_cmd=None)
Gain_L2 = Parameter('Gain_L2', label='Gain L2', unit='', set_cmd=None, get_cmd=None)

#Add components
station.add_component(Irange)
station.add_component(Vrange)
station.add_component(TGrange)
station.add_component(TGrange2)

station.add_component(Irange_division)
station.add_component(Vrange_division)
station.add_component(TGrange_division)
station.add_component(TGrange_division2)

station.add_component(Irange_AC_division)
station.add_component(Vrange_AC_division)
                             
station.add_component(Gain_K1)
station.add_component(Gain_K2)
station.add_component(Gain_L1)
station.add_component(Gain_L2)
print('\nStation list:')
pprint(station.components)


Station list:
{'DMM6500': <Keithley_6500: DMM6500>,
 'Gain_K1': <qcodes.parameters.parameter.Parameter: Gain_K1 at 2694746060016>,
 'Gain_K2': <qcodes.parameters.parameter.Parameter: Gain_K2 at 2694746060736>,
 'Gain_L1': <qcodes.parameters.parameter.Parameter: Gain_L1 at 2694745746880>,
 'Gain_L2': <qcodes.parameters.parameter.Parameter: Gain_L2 at 2694745745872>,
 'Irange': <qcodes.parameters.parameter.Parameter: Irange at 2694332954656>,
 'Irange_AC_division': <qcodes.parameters.parameter.Parameter: Irange_AC_division at 2694754651440>,
 'Irange_division': <qcodes.parameters.parameter.Parameter: Irange_division at 2694745834064>,
 'K2000': <Keithley2000: K2000>,
 'TGrange': <qcodes.parameters.parameter.Parameter: TGrange at 2694332955040>,
 'TGrange2': <qcodes.parameters.parameter.Parameter: TGrange2 at 2694745835696>,
 'TGrange_division': <qcodes.parameters.parameter.Parameter: TGrange_division at 2694754650384>,
 'TGrange_division2': <qcodes.parameters.parameter.Parameter: TGrang

## Parameters - update here during measuerements

In [17]:
Irange.set(100e-9) #Unit Amp V=IR, 1V= 1e-6 x 1MOhm=1e-7 x 10MOhm
Vrange.set(1e-3)
TGrange.set(15)
TGrange2.set(15)
Gain_K1.set(100)
Gain_K2.set(100)
# Gain_K3.set(1e2)
Gain_L1.set(Gain_K1())
Gain_L2.set(Gain_K2())

## DC parameters

In [10]:
appl_current = ScaledParameter(ivvi.dac1, division = Irange_division, name = 'appl_current', unit = 'A')
appl_voltage = ScaledParameter(ivvi.dac2, division = Vrange_division, name = 'appl_voltage', unit = 'V')
# appl_TG = ScaledParameter(ivvi.dac3, division = TGrange_division, name = 'appl_TG', unit = 'V')
# appl_TG2 = ScaledParameter(ivvi.dac4, division = TGrange_division, name = 'appl_TG2', unit = 'V')
# appl_TG7 = ScaledParameter(ivvi.dac7, division = TGrange_division2, name = 'appl_TG7', unit = 'V')
# appl_TG9 = ScaledParameter(ivvi.dac9, division = TGrange_division2, name = 'appl_TG9', unit = 'V')
# appl_TG11 = ScaledParameter(ivvi.dac11, division = TGrange_division2, name = 'appl_TG11', unit = 'V')

# meas_voltage_K1 = ScaledParameter(DMM6500.amplitude, division = Gain_K1, name = 'meas_voltage_K1', unit = 'V')
meas_voltage_K2 = ScaledParameter(DMM6500.amplitude, division = Gain_K2, name = 'meas_voltage_K2', unit = 'V')

# meas_leakage = ScaledParameter(DMM6500.amplitude, division = Gain_K2, name = 'meas_leakage', unit = 'A')

# meas_current_K1 = ScaledParameter(DMM6500.amplitude, division = Gain_K1, name = 'meas_current_K1', unit = 'A')
meas_current_K2 = ScaledParameter(DMM6500.amplitude, division = Gain_K2, name = 'meas_current_K2', unit = 'A')

station.add_component(appl_current)
station.add_component(appl_voltage)
# station.add_component(appl_TG)
# station.add_component(appl_TG2)
# station.add_component(appl_TG7)
# station.add_component(appl_TG9)
# station.add_component(appl_TG11)
# station.add_component(meas_voltage_K1)
station.add_component(meas_voltage_K2)
# station.add_component(meas_leakage)
# station.add_component(meas_current_K1)
station.add_component(meas_current_K2)

'meas_current_K2'

In [75]:
ivvi.dac1(1000)

In [33]:
appl_current(0)

In [ ]:
I=1e-6
appl_current(I)
sleep(1)
V=meas_voltage_K2()
R=V/I
print(R)

##  AC parameters

In [ ]:
appl_current_AC = ScaledParameter(lockin_1.amplitude, division = Irange_AC_division, name = 'appl_current_AC', unit = 'A')
appl_voltage_AC = ScaledParameter(lockin_1.amplitude, division = Vrange_AC_division, name = 'appl_voltage_AC', unit = 'V')

station.add_component(appl_current_AC)
station.add_component(appl_voltage_AC)

meas_voltage_AC_L1 = ScaledParameter(lockin_1.X, division = Gain_L1, name = 'meas_voltage_Lockin1', unit = 'V')
meas_current_AC_L1 = ScaledParameter(lockin_1.X, division = Gain_L1, name = 'meas_current_Lockin1', unit = 'A')
meas_phase_AC_L1 = ScaledParameter(lockin_1.P, division = 1, name = 'meas_phase_Lockin1', unit = 'degree')


# meas_voltage_AC_L2 = ScaledParameter(lockin_2.X, division = Gain_L2, name = 'meas_voltage_Lockin2', unit = 'V')
# meas_current_AC_L2 = ScaledParameter(lockin_2.X, division = Gain_L2, name = 'meas_current_Lockin2', unit = 'A')
# meas_phase_AC_L2 = ScaledParameter(lockin_2.P, division = 1, name = 'meas_phase_Lockin1', unit = 'degree')

station.add_component(meas_voltage_AC_L1)
station.add_component(meas_current_AC_L1)
station.add_component(meas_phase_AC_L1)

# station.add_component(meas_voltage_AC_L2)
# station.add_component(meas_current_AC_L2)

meas_voltage_AC_L1_R = ScaledParameter(lockin_1.R, division = Gain_L1, name = 'meas_voltage_R_Lockin1', unit = 'V')
# meas_voltage_AC_L2_R = ScaledParameter(lockin_2.R, division = Gain_L2, name = 'meas_voltage_R_Lockin2', unit = 'V')

station.add_component(meas_voltage_AC_L1_R)
# station.add_component(meas_voltage_AC_L2_R)

In [ ]:
meas_voltage_AC_L1_P = ScaledParameter(lockin_1.P, division = 1, name = 'meas_voltage_Lockin1_P', unit = 'degree')
meas_current_AC_L1_P = ScaledParameter(lockin_1.P, division = 1, name = 'meas_current_Lockin1_P', unit = 'degree')

# meas_voltage_AC_L2_P = ScaledParameter(lockin_2.P, division = 1, name = 'meas_voltage_Lockin2_P', unit = 'degree')
# meas_current_AC_L2_P = ScaledParameter(lockin_2.P, division = 1, name = 'meas_current_Lockin2_P', unit = 'degree')

station.add_component(meas_voltage_AC_L1_P)
station.add_component(meas_current_AC_L1_P)

# station.add_component(meas_voltage_AC_L2_P)
# station.add_component(meas_current_AC_L2_P)

## Another useful example: magnet using dac (Janis)

>2231.2 G per A => 0.22312 T per A<br>
I source 10mA per V => 1e-5 A per 1mV<br>
result: 0.22312 T/A * 1e-5 A/mV

In [12]:
# def get_R_from_IV(dataid):
#     dataset = qc.load_by_id(dataid)
#     voltage = dataset.get_parameter_data()['meas_voltage_K2']['meas_voltage_K2']
#     current = dataset.get_parameter_data()['meas_voltage_K2']['appl_current']
#     fit_values = np.polyfit(current,voltage,1)
    
#     return fit_values[0]

dataset = qc.load_by_id(1)

def get_R_from_IV_2(dataid):
    dataset = qc.load_by_id(dataid)
    voltage = dataset.get_parameter_data()['meas_voltage_K2']['meas_voltage_K2']
    current = dataset.get_parameter_data()['meas_voltage_K2']['appl_current']
    fit_values = np.polyfit(current,voltage,1)

    return fit_values[0]

def get_R_from_IV(dataid):
    dataset = qc.load_by_id(dataid)
    para_list = dataset.parameters.split(",")
    voltage = dataset.get_parameter_data()[para_list[1]][para_list[1]]
    current = dataset.get_parameter_data()[para_list[1]][para_list[0]]
    fit_values = np.polyfit(current,voltage,1)
    return fit_values[0]


def get_R_from_IV_K2(dataid):
    dataset = qc.load_by_id(dataid)
    voltage = dataset.get_parameter_data()['meas_voltage_K2']['meas_voltage_K2']
    current = dataset.get_parameter_data()['meas_voltage_K2']['appl_current']
    fit_values = np.polyfit(current,voltage,1)
    return fit_values[0]

def autosave_plot_by_id(run_id, save_path):
    fig, ax = plt.subplots()
    plot_by_id(run_id, axes=ax)
    plt.savefig(save_path)
    plt.close()

def get_detaset_info(dataset=dataset):
    run_id = dataset.captured_run_id
    exp_name = dataset.exp_name
    sample_name = dataset.sample_name
    name = dataset.name
    description = dataset.description
    sample_junction = extract_sample_junction(sample_name)
    return run_id, exp_name, sample_name, sample_junction, name, description

def extract_sample_junction(sample_name):
    """Extracts the complete sample junction from the provided sample name.

    Args:
      sample_name: The name of the sample containing the junction information.

    Returns:
      The extracted sample junction, or None if not found.
    """

    delimiter = "_J"
    if delimiter in sample_name:
        # +3 to include "_J"
        return sample_name[:sample_name.find(delimiter) + 3]
    else:
        return None

def IV(run_id=1):
    dataset = qc.load_by_id(run_id)
    R = get_R_from_IV(run_id)
    run_id, exp_name, sample_name, sample_junction, name, description = get_detaset_info(
        dataset)
    df = dataset.to_pandas_dataframe().reset_index()
    para_list = dataset.parameters.split(",")

    # Create a Plotly trace
    trace = go.Scatter(x=df[para_list[0]], y=df[para_list[1]]*1e3, mode='lines', name='Voltage vs. Current')

    # Create layout with micro-scale y-axis
    layout = go.Layout(title=f'<b>#{run_id} {exp_name}</b> {sample_name}, R_fit={R:.4f} Ω', title_font=dict(size=28),
                       xaxis=dict(title=para_list[0], tickfont=dict(size=16),title_font=dict(size=20)),
                       yaxis=dict(title=f"{para_list[1]} mV", tickfont=dict(size=16),title_font=dict(size=20)), 
                       width=1080, height=960,
                       margin=dict(l=10, r=10, t=50, b=10))

    # Create the figure
    fig = go.Figure(data=[trace], layout=layout)
    
    fig.show()

def IV_diff(run_id=1):
    import numpy as np
    import plotly.graph_objs as go
    from scipy.signal import find_peaks
    dataset = qc.load_by_id(run_id)

    df = dataset.to_pandas_dataframe().reset_index()
    para_list = dataset.parameters.split(",")

    current = df[para_list[0]].tolist()
    voltage = df[para_list[1]].tolist()

    # Compute differential of voltage with respect to current
    differential_voltage = np.diff(voltage) / np.diff(current)

    # Since the differential will have one less data point, adjust the current data
    # to have the same length as the differential data
    current = current[:-1]

    # Find peaks in the differential signal
    peaks, _ = find_peaks(differential_voltage)

    # Extract peak current and peak differential voltage values
    peak_current = [current[i] for i in peaks]
    peak_voltage_diff = [differential_voltage[i] for i in peaks]

    # Sort the peaks based on their values
    sorted_peak_indices = np.argsort(peak_voltage_diff)[::-1]  # Sort in descending order
    top_two_peak_indices = sorted_peak_indices[:2]  # Select the top two indices

    # Extract the top two peaks
    top_two_peak_current = [peak_current[i] for i in top_two_peak_indices]
    top_two_peak_voltage_diff = [peak_voltage_diff[i] for i in top_two_peak_indices]

    # Calculate the center points of the top two peaks
    center_points = [(top_two_peak_current[i] + top_two_peak_current[i+1]) / 2 for i in range(len(top_two_peak_current) - 1)]

    # Calculate the difference between the top two biggest peaks
    peak_difference = abs(top_two_peak_current[0] - top_two_peak_current[1])

    # Determine the appropriate unit for current
    if peak_difference >= 1e-3:
        unit = "mA"
        conversion_factor = 1e3
    elif peak_difference >= 1e-6:
        unit = "μA"
        conversion_factor = 1e6
    else:
        unit = "nA"
        conversion_factor = 1e9

    # Determine the appropriate unit for current
    if peak_difference >= 1e-3:
        unit = "mA"
    elif peak_difference >= 1e-6:
        unit = "μA"
    elif peak_difference >= 1e-9:
        unit = "nA"
    else:
        unit = "pA"

    # Create a Plotly trace for the differential
    trace_diff = go.Scatter(x=[c * conversion_factor for c in current], y=differential_voltage, mode='lines', name='d(Voltage)/d(Current)')
    # Add markers for the top two peaks
    trace_top_two_peaks = go.Scatter(x=[p * conversion_factor for p in top_two_peak_current], y=top_two_peak_voltage_diff, mode='markers', marker=dict(color='red'), name='Top Two Peaks')
    # Add markers for the center points
    trace_center_points = go.Scatter(x=center_points, y=[0]*len(center_points), mode='markers', marker=dict(color='blue', symbol='cross'), name='Center Points')

    # Create layout
    layout = go.Layout(title=f'Run #{id} I vs. dV/dI',
                    xaxis=dict(title=f'Current ({unit})'),
                    yaxis=dict(title='d(Voltage)/d(Current)'),
                    legend=dict(orientation="h", yanchor="top", y=1.10, xanchor="center", x=0.5),
                    width=600, height=600  
                    )

    # Create the figure
    fig = go.Figure(data=[trace_diff, trace_top_two_peaks, trace_center_points], layout=layout)

    # Add annotation for the current value of center points
    for i, center_point in enumerate(center_points):
        fig.add_annotation(
            x=center_point * conversion_factor,
            y=0, #0.4 * max(differential_voltage),
            text=f'Center Point: {center_point * conversion_factor:.3f}{unit} <br> Peak difference: {peak_difference*conversion_factor:.3f} {unit}',
            showarrow=True,
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor='black',
            ax=-100,
            ay=-200 - 30 * i
        )

    # Plot the figure
    fig.show()

def get_Mag(show=True):
    Mag_x = magnet_x.field()
    Mag_y = magnet_y.field()
    Mag_z = magnet_z.field()
    if show:
        print(f"Mag_x: {Mag_x*1e3:.6f} mT")
        print(f"Mag_y: {Mag_y*1e3:.6f} mT")
        print(f"Mag_z: {Mag_z*1e3:.6f} mT")
    else: 
        return Mag_x, Mag_y, Mag_z

In [ ]:
appl_current(0.5e-6)
appl_current_AC(10e-9)

In [ ]:
df = dataset.to_pandas_dataframe().reset_index()
df

# Simple IVs

## IV

In [35]:
ivvi.dac1.step = 100
ivvi.dac1.inter_delay = 0.1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
import matplotlib.font_manager as fm

# 設置支持中文的字體
plt.rcParams['font.sans-serif'] = ['simsun']  # 使用宋體字
plt.rcParams['axes.unicode_minus'] = False  # 解決負號顯示問題

# 確認字體是否已經加載
print(fm.findSystemFonts(fontpaths=None, fontext='ttf'))

# 電流與電壓數據
dataset = qc.load_by_id(dataid)
voltage = dataset.get_parameter_data()['meas_voltage_K2']['meas_voltage_K2']
current = dataset.get_parameter_data()['meas_voltage_K2']['appl_current']

# 計算電阻 (線性擬合)
slope, intercept, r_value, p_value, std_err = linregress(current, voltage)
resistance = slope  # 斜率即為電阻

# 計算擬合值與殘差
voltage_fit = slope * current + intercept
residuals = voltage - voltage_fit

# 計算信號與噪聲功率
signal_power = np.var(voltage_fit)
noise_power = np.var(residuals)

# 計算 SNR
snr = 10 * np.log10(signal_power / noise_power)

# 輸出結果
print(f"電阻 R: {resistance:.4f} Ohm")
print(f"截距 Intercept: {intercept:.4e} V")
print(f"相關係數 R-value: {r_value:.4f}")
print(f"信噪比 SNR: {snr:.2f} dB")

# 繪製量測曲線及擬合曲線
plt.figure(figsize=(10, 6))
plt.plot(current, voltage, 'o', label='測量數據')
plt.plot(current, voltage_fit, '-', label=f'線性擬合\n斜率: {slope:.4f} Ohm\n截距: {intercept:.4e} V\nR-value: {r_value:.4f}')
plt.xlabel('電流 (A)')
plt.ylabel('電壓 (V)')
plt.title('I-V 曲線及線性擬合')
plt.legend()
plt.grid(True)
plt.show()

# 計算微分電阻
differential_voltage = np.diff(voltage) / np.diff(current)
differential_current = (current[:-1] + current[1:]) / 2  # 使用中間點作為電流值

# 計算微分電阻的信號與噪聲
signal = np.mean(differential_voltage)
noise_std = np.std(differential_voltage)
snr = signal / noise_std
snr_db = 20 * np.log10(snr)

# 輸出微分電阻的 SNR 結果
print(f"微分電阻信號: {signal}")
print(f"微分電阻噪聲標準差: {noise_std}")
print(f"微分電阻 SNR: {snr}")
print(f"微分電阻 SNR (dB): {snr_db}")

# 計算微分電阻擬合曲線
diff_slope, diff_intercept, diff_r_value, diff_p_value, diff_std_err = linregress(differential_current, differential_voltage)
differential_voltage_fit = diff_slope * differential_current + diff_intercept

# 繪製微分電阻曲線及擬合曲線
plt.figure(figsize=(10, 6))
plt.plot(differential_current, differential_voltage, 'o', label='微分電阻')
plt.plot(differential_current, differential_voltage_fit, '-', label=f'微分電阻擬合\n斜率: {diff_slope:.4f} Ohm\n截距: {diff_intercept:.4e} V\nR-value: {diff_r_value:.4f}')
plt.xlabel('電流 (A)')
plt.ylabel('微分電壓 (V)')
plt.title('微分電阻曲線及擬合')
plt.legend()
plt.grid(True)
plt.show()

In [19]:
appl_current(-100e-9)
sleep(1)

# exp = qc.new_experiment('IV', sample_name='018_1kOhm')
exp = qc.new_experiment('IV', sample_name='018-02_24-25-48-47')

dataid = do1d(appl_current, -100e-9 ,100e-9, 101, 0.01,meas_voltage_K2,)
# dataid = do1d(appl_current, 6e-6 , -6e-6, 251, 0.01, meas_voltage_K1)
# ivvi.set_dacs_zero()

appl_current(0)
# plot = plot_by_id(dataid)
print(f"R = {get_R_from_IV(dataid)}")
IV(dataid)
# IV_diff(dataid)

Starting experimental run with id: 18. 


100%|██████████| 101/101 [00:28<00:00,  3.53it/s]


R = 132231.96569132208


In [45]:
IV(6)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
import matplotlib.font_manager as fm

# 設置支持中文的字體
plt.rcParams['font.sans-serif'] = ['simsun']  # 使用宋體字
plt.rcParams['axes.unicode_minus'] = False  # 解決負號顯示問題

# 確認字體是否已經加載
print(fm.findSystemFonts(fontpaths=None, fontext='ttf'))

# 電流與電壓數據
dataset = qc.load_by_id(dataid)
voltage = dataset.get_parameter_data()['meas_voltage_K2']['meas_voltage_K2']
current = dataset.get_parameter_data()['meas_voltage_K2']['appl_current']

# 計算電阻 (線性擬合)
slope, intercept, r_value, p_value, std_err = linregress(current, voltage)
resistance = slope  # 斜率即為電阻

# 計算擬合值與殘差
voltage_fit = slope * current + intercept
residuals = voltage - voltage_fit

# 計算信號與噪聲功率
noise_std = np.var(residuals)

# 計算 SNR
snr_mean = np.mean(voltage_fit**2)/noise_std
snr_mean_db = 10 * np.log10(snr_mean)
snr_fit = voltage_fit**2/noise_std**2
snr_fit_db = 10 * np.log10(snr_fit)



# 計算信號功率
P_signal = np.mean(signal) ** 2

# 使用變異數計算 SNR
P_noise_var = np.var(noise)
SNR_var = 10 * np.log10(P_signal / P_noise_var)

# 使用標準差計算 SNR
P_noise_sd = np.std(noise) ** 2  # 注意標準差需平方
SNR_sd = 10 * np.log10(P_signal / P_noise_sd)

print(f"SNR 使用變異數計算: {SNR_var:.2f} dB")
print(f"SNR 使用標準差計算: {SNR_sd:.2f} dB")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
import matplotlib.font_manager as fm

# 設置支持中文的字體
plt.rcParams['font.sans-serif'] = ['simsun']  # 使用宋體字
plt.rcParams['axes.unicode_minus'] = False  # 解決負號顯示問題

# 確認字體是否已經加載
print(fm.findSystemFonts(fontpaths=None, fontext='ttf'))

# 電流與電壓數據
dataset = qc.load_by_id(dataid)
voltage = dataset.get_parameter_data()['meas_voltage_K2']['meas_voltage_K2']
current = dataset.get_parameter_data()['meas_voltage_K2']['appl_current']

# 計算電阻 (線性擬合)
slope, intercept, r_value, p_value, std_err = linregress(current, voltage)
resistance = slope  # 斜率即為電阻

# 計算擬合值與殘差
voltage_fit = slope * current + intercept
residuals = voltage - voltage_fit

# 計算信號與噪聲功率
signal_power = np.var(voltage_fit)
noise_power = np.var(residuals)

# 計算 SNR
snr = 10 * np.log10(signal_power / noise_power)

# 輸出結果
print(f"電阻 R: {resistance:.4f} Ohm")
print(f"截距 Intercept: {intercept:.4e} V")
print(f"相關係數 R-value: {r_value:.4f}")
print(f"信噪比 SNR: {snr:.2f} dB")

# 計算微分電阻
differential_voltage = np.diff(voltage) / np.diff(current)
differential_current = (current[:-1] + current[1:]) / 2  # 使用中間點作為電流值

# 計算微分電阻的信號與噪聲
signal = np.mean(differential_voltage)
noise_std = np.std(differential_voltage)
snr = signal / noise_std
snr_db = 20 * np.log10(snr)

# 輸出微分電阻的 SNR 結果
print(f"微分電阻信號: {signal}")
print(f"微分電阻噪聲標準差: {noise_std}")
print(f"微分電阻 SNR: {snr}")
print(f"微分電阻 SNR (dB): {snr_db}")

# 計算微分電阻擬合曲線
diff_slope, diff_intercept, diff_r_value, diff_p_value, diff_std_err = linregress(differential_current, differential_voltage)
differential_voltage_fit = diff_slope * differential_current + diff_intercept

# 繪製 I-V 曲線及微分電阻曲線
fig, ax1 = plt.subplots(figsize=(12, 8))

# 繪製 I-V 曲線及擬合曲線
ax1.plot(current, voltage, 'o', label='測量數據')
ax1.plot(current, voltage_fit, '-', label=f'線性擬合\n斜率: {slope:.4f} Ohm\n截距: {intercept:.4e} V\nR-value: {r_value:.4f}')
ax1.set_xlabel('電流 (A)')
ax1.set_ylabel('電壓 (V)')
ax1.set_title('I-V 曲線及微分電阻曲線')
ax1.legend(loc='upper left')
ax1.grid(True)

# 繪製微分電阻曲線及擬合曲線
ax2 = ax1.twinx()
ax2.plot(differential_current, differential_voltage, 'o', color='tab:orange', label='微分電阻')
ax2.plot(differential_current, differential_voltage_fit, '-', color='tab:red', label=f'微分電阻擬合\n斜率: {diff_slope:.4f} Ohm\n截距: {diff_intercept:.4e} V\nR-value: {diff_r_value:.4f}')
ax2.set_ylabel('微分電壓 (V)')
ax2.legend(loc='upper right')

plt.show()

import numpy as np

# 假設的信號和雜訊
signal = np.array([1.0, 1.1, 1.2, 0.9, 1.0])  # 信號值
noise = np.array([0.05, -0.02, 0.03, -0.04, 0.01])  # 雜訊值

# 計算信號功率
P_signal = np.mean(signal) ** 2

# 使用變異數計算 SNR
P_noise_var = np.var(noise)
SNR_var = 10 * np.log10(P_signal / P_noise_var)

# 使用標準差計算 SNR
P_noise_sd = np.std(noise) ** 2  # 注意標準差需平方
SNR_sd = 10 * np.log10(P_signal / P_noise_sd)

print(f"SNR 使用變異數計算: {SNR_var:.2f} dB")
print(f"SNR 使用標準差計算: {SNR_sd:.2f} dB")

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 創建子圖，包含雙 Y 軸
fig = make_subplots(specs=[[{"secondary_y": True}]])

# 繪製 I-V 曲線及擬合曲線
fig.add_trace(
    go.Scatter(
        x=current,
        y=voltage,
        mode='markers',
        name='測量數據',
        marker=dict(color='blue')
    ),
    secondary_y=False
)

fig.add_trace(
    go.Scatter(
        x=current,
        y=voltage_fit,
        mode='lines',
        name=f'線性擬合<br>斜率: {slope:.4f} Ohm<br>截距: {intercept:.4e} V<br>R-value: {r_value:.4f}',
        line=dict(color='blue', dash='dash')
    ),
    secondary_y=False
)

# 繪製微分電阻曲線及擬合曲線
fig.add_trace(
    go.Scatter(
        x=differential_current,
        y=differential_voltage,
        mode='markers',
        name='微分電阻',
        marker=dict(color='orange')
    ),
    secondary_y=True
)

fig.add_trace(
    go.Scatter(
        x=differential_current,
        y=differential_voltage_fit,
        mode='lines',
        name=f'微分電阻擬合<br>斜率: {diff_slope:.4f} Ohm<br>截距: {diff_intercept:.4e} V<br>R-value: {diff_r_value:.4f}',
        line=dict(color='red', dash='dash')
    ),
    secondary_y=True
)

# 更新圖表的佈局
fig.update_layout(
    title='I-V 曲線及微分電阻曲線',
    xaxis=dict(title='電流 (A)'),
    yaxis=dict(title='電壓 (V)', titlefont=dict(color='blue')),
    yaxis2=dict(title='微分電壓 (V)', titlefont=dict(color='orange')),
    legend=dict(x=0.01, y=0.99, bgcolor='rgba(255,255,255,0.7)', bordercolor='black', borderwidth=1),
    width=800,
    height=600
)

# 顯示圖表
fig.show()


In [18]:
K2440.output(1)
DMM6500.averaging_count(5)
DMM6500.averaging_enabled(True)
DMM6500.averaging_enabled(False)
K2440.output(0)

In [19]:
# Set current to 0
K2440.output(0)
appl_current(0e-6)

In [ ]:
IV(3)

In [ ]:
I_range=1e-6
K2440.output(1)
appl_current(-I_range) #Limit:1e-5 Amp(1MOhms risistor),1e-6 Amp(10MOhms risistor)
sleep(2)
# station.snapshot(update=True);
exp = qc.new_experiment('I-V', sample_name='009-1_05-04-45-44G')
dataid = do1d(appl_current, -I_range ,I_range, 501, 0.03, meas_voltage_K1)
appl_current(0)
K2440.output(0)

plot_by_id(dataid)
#save_path = r"C:\qcodes\personal_scripts\Albert\104\Plot\104_{}.png".format(dataid)  # Path to save the plot image
#autosave_plot_by_id(dataid, save_path)
print(f"R = {get_R_from_IV(dataid)}")
# dataset = qc.load_by_id(dataid)
IV(dataid)
IV_diff(dataid)

In [ ]:
IV(4)

In [ ]:
get_R_from_IV(dataid)

In [ ]:
dataset = qc.load_by_id(dataid)
get_R_from_IV(dataid)
IV(dataid)
IV_diff(dataid)

In [32]:
import plotly.graph_objs as go

In [ ]:
for i in tqdm(range(1000)):
    K2440.output(1)
    I_range=1e-6
    appl_current(-I_range) #Limit:1e-5 Amp(1MOhms risistor),1e-6 Amp(10MOhms risistor)
    sleep(0.001)
    # station.snapshot(update=True);
    exp = qc.new_experiment('I-V', sample_name=f'103-03_J1_33-32-30-31_1M_F{i+1}')
    dataid = do1d(appl_current, -I_range ,I_range, 101, 0.0001, meas_voltage_K2, meas_voltage_AC_L1,meas_voltage_AC_L1_P)
    exp = qc.new_experiment('I-V', sample_name=f'103-03_J1_33-32-30-31_1M_B{i+1}')
    dataid = do1d(appl_current, I_range ,-I_range, 101, 0.0001, meas_voltage_K2)
    K2440.output(0)
    clear_output(True)

In [ ]:
get_Mag()

In [ ]:
SGS.off()

In [ ]:
SGS.power(0)

In [ ]:
DMM6500.averaging_count(5)
DMM6500.averaging_enabled(True)

In [ ]:
for i in tqdm(range(21)):
    I_range = round(round(0.55 + 5*i * 1e-4, 6)* 1e-6, 10)  # Round to 3 decimal places
    print(I_range)


In [ ]:
for i in tqdm(range(21)):
    K2440.output(1)
    I_range = round(round(0.55 + 5*i * 1e-4, 6)* 1e-6, 10) 
    appl_current(-I_range) #Limit:1e-5 Amp(1MOhms risistor),1e-6 Amp(10MOhms risistor)
    sleep(2)
    # station.snapshot(update=True);
    exp = qc.new_experiment('I-V', sample_name=f'103-04_J1_24-25-05-04(06-07G)_1M_By@0mT_{-I_range}_201')
    dataid = do1d(appl_current, -I_range ,I_range, 201, 0.00001, meas_voltage_K2)

    appl_current(0)
    K2440.output(0)


    plot_by_id(dataid)
    save_path = r"D:\data\Albert\103\Plot\104_{}.png".format(dataid)  # Path to save the plot image
    autosave_plot_by_id(dataid, save_path)
    print(f"ID:{dataid}")
    print(f"R = {get_R_from_IV_2(dataid)}")
    dataset = qc.load_by_id(dataid)
    IV(id=dataid)
    IV_diff(id=dataid)

In [ ]:
DMM6500.averaging_count(5)
DMM6500.averaging_enabled(True)

In [ ]:
K2440.output(1)
I_range = -6e-6
appl_current(-I_range) #Limit:1e-5 Amp(1MOhms risistor),1e-6 Amp(10MOhms risistor)
sleep(2)
# station.snapshot(update=True);
exp = qc.new_experiment('I-V', sample_name=f'103-03_J2_27-26-30-31M(32G-33G-28G-29G)_1M_By@0.1mT{-I_range}')
dataid = do1d(appl_current, -I_range  ,I_range , 2601, 0.00001, meas_voltage_K2)
appl_current(0)
K2440.output(0)
plot_by_id(dataid)
save_path = r"D:\data\Albert\103\Plot\104_{}.png".format(dataid)  # Path to save the plot image
autosave_plot_by_id(dataid, save_path)
print(f"R = {get_R_from_IV_2(dataid)}")
dataset = qc.load_by_id(dataid)

In [ ]:
K2440.output(1)
I_range = 6e-6
appl_current(-I_range) #Limit:1e-5 Amp(1MOhms risistor),1e-6 Amp(10MOhms risistor)
sleep(2)
# station.snapshot(update=True);
exp = qc.new_experiment('I-V', sample_name=f'103-03_J2_27-26-30-31M(32G-33G-28G-29G)_1M_By@0.1mT{-I_range}')
dataid = do1d(appl_current, -I_range  ,I_range , 2601, 0.00001, meas_voltage_K2)
appl_current(0)
K2440.output(0)
plot_by_id(dataid)
save_path = r"D:\data\Albert\103\Plot\104_{}.png".format(dataid)  # Path to save the plot image
autosave_plot_by_id(dataid, save_path)
print(f"ID:{dataid}")
print(f"R = {get_R_from_IV_2(dataid)}")
dataset = qc.load_by_id(dataid)
# IV(id=dataid)
# IV_diff(id=dataid)

In [ ]:
25-05-06-07(04-24G)


In [ ]:
K2440.output(1)
I_range=-7e-6
appl_current(-I_range) #Limit:1e-5 Amp(1MOhms risistor),1e-6 Amp(10MOhms risistor)
sleep(2)
# station.snapshot(update=True);
exp = qc.new_experiment('I-V', sample_name='103-03_J2_32-30-26-27_1M_By@0mT')
dataid = do1d(appl_current, -I_range ,I_range, 1401, 0.0001, meas_voltage_K2)

appl_current(0)
K2440.output(0)


plot_by_id(dataid)
save_path = r"D:\data\Albert\103\Plot\104_{}.png".format(dataid)  # Path to save the plot image
autosave_plot_by_id(dataid, save_path)
print(f"ID:{dataid}")
print(f"R = {get_R_from_IV_2(dataid)}")
dataset = qc.load_by_id(dataid)
IV(id=dataid)
IV_diff(id=dataid)

In [ ]:
K2440.output(1)
I_range=7e-6
appl_current(-I_range) #Limit:1e-5 Amp(1MOhms risistor),1e-6 Amp(10MOhms risistor)
sleep(2)
# station.snapshot(update=True);
exp = qc.new_experiment('I-V', sample_name='103-03_J2_32-30-26-27_1M_By@0mT')
dataid = do1d(appl_current, -I_range ,I_range, 1401, 0.0001, meas_voltage_K2)

appl_current(0)
K2440.output(0)


plot_by_id(dataid)
save_path = r"D:\data\Albert\103\Plot\104_{}.png".format(dataid)  # Path to save the plot image
autosave_plot_by_id(dataid, save_path)
print(f"ID:{dataid}")
print(f"R = {get_R_from_IV_2(dataid)}")
dataset = qc.load_by_id(dataid)
IV(id=dataid)
IV_diff(id=dataid)

In [ ]:
# Set current to 0
K2440.output(0)
appl_current(0e-6)

In [ ]:
K2440.output(1)
appl_current(0.1e-7)

In [ ]:
runid=6
print(get_R_from_IV_2(runid))

## 加TG (1D)

In [ ]:
K2440.output(1)
appl_TG(0)

In [ ]:
K2440.output(1)
K2440.output(1)
appl_TG(-50)
sleep(3)
appl_current(-1.5e-8)
sleep(1)

# station.snapshot(update=True);
exp = qc.new_experiment('IV', sample_name='TG21')

#do1d(param_set, start, stop, num_points, delay, *param_meas)
dataid = do1d(appl_current, -1.5e-8 ,1.5e-8, 101, 0.05, meas_voltage_K2)
# dataid = do1d(appl_current, 6e-6 , -6e-6, 251, 0.01, meas_voltage_K1)
# ivvi.set_dacs_zero()

K2440.output(0)
K2440.output(0)
appl_current(0)
plot_by_id(dataid)
#print(get_R_from_IV(dataid))
# print(get_R_from_IV_2(dataid))


## 歸零


In [ ]:
#K2440.output(1)
K2440.output(1)

appl_current(0)
sleep(1)
#appl_TG(0)
sleep(5)

#K2440.output(0)
K2440.output(0)

# Magnetic field

## IV vs Bx

In [ ]:
K2440.rangev()

In [ ]:
K2440.volt(0)
K2440.output(1) #output off
sleep(1)
appl_current(1.5e-7)
sleep(2)
meas_voltage_K2()

In [ ]:
sleep(120)

In [ ]:
def get_Mag(show=True):
    Mag_x = magnet_x.field()
    Mag_y = magnet_y.field()
    Mag_z = magnet_z.field()
    if show:
        print(f"Mag_x: {Mag_x*1e3:.6f} mT")
        print(f"Mag_y: {Mag_y*1e3:.6f} mT")
        print(f"Mag_z: {Mag_z*1e3:.6f} mT")
    else: 
        return Mag_x, Mag_y, Mag_z
get_Mag()

In [ ]:
get_Mag()

In [ ]:
magnet_y.field()

In [ ]:
for i in tqdm(range(23), desc="First loop"):
    value = (25 - i) * 1e-3 
    value = float(f"{value:.10f}")
    print(value)

In [ ]:
from tqdm import tqdm
import time

from IPython.display import clear_output
# First loop
for i in tqdm(range(6), desc="First loop"):
    Mag_x = magnet_x.field()
    value = (15 + i) * 1e-3
    value = float(f"{value:.10f}")
    magnet_x.field(value)
    print(f"magnet_x is {Mag_x:.8f} T")
    time.sleep(15)
    clear_output(True)

# Pause for 30 seconds
time.sleep(60)

# Second loop
for i in tqdm(range(23), desc="First loop"):
    Mag_x = magnet_x.field()
    value = (25 - i) * 1e-3 
    value = float(f"{value:.10f}")
    magnet_x.field(value)
    print(f"magnet_x is {Mag_x:.8f} T")
    time.sleep(15)
    clear_output(True)

# Pause for 180 seconds
time.sleep(300)

In [ ]:
magnet_x.field(-20)

In [ ]:
Mag = -20e-3
I_range=2.5e-6
K2440.output(1)
sleep(2)
magnet_x.field(Mag)
sleep(10)
appl_current(-I_range)
sleep(3)
direction = 'backward' if Mag > 0 else 'forward'


# station.snapshot(update=True);
exp = qc.new_experiment('IV vs Bx', sample_name=f'103-06_J2_12-13-18-19_1M_Bx_{direction}_By@0mT')

dataid = do2d(magnet_x.field, Mag, -Mag, 76 , 0.7, appl_current, -I_range ,I_range, 101, 0.01, meas_voltage_K2)

appl_current(0)
K2440.output(0)
# magnet_y.field(0)

plot_by_id(dataid)
autosave_plot_by_id(dataid, save_path)

In [ ]:
K2440.output(1)
sleep(2)
Mag = 10e-3
I_range=2.5e-5

magnet_x.field(Mag)
sleep(30)
appl_current(-I_range)
sleep(3)
direction = 'backward' if Mag > 0 else 'forward'


# station.snapshot(update=True);

exp = qc.new_experiment('IV vs Bx', sample_name=f'094-20IV_Bx_16-17-19-18_10M_{direction}')

dataid = do2d(magnet_x.field, Mag, -Mag, 101 , 1.5, appl_current, -I_range ,I_range, 101, 0.05, meas_voltage_K2)#M:I Delay time 10:1


appl_current(0)
K2440.output(0)
# magnet_x.field(0)

plot_by_id(dataid)
autosave_plot_by_id(dataid, save_path)

In [ ]:
set_magnetic_field_stepwise(magnet=magnet_x, current_field=-40e-3, target_field=-18e-3, step_size=1e-3, delay=2.5)

In [ ]:
def set_magnetic_field_stepwise(magnet=magnet_x, current_field=0, target_field=-10e-3, step_size=1e-3, delay=2.5):
    
    """Set the magnetic field with anti-hysteresis mechanism witch will go to target magnetic field +/- 5mT, 
    if the target magnetic field is greater/less than the current magnetic field, 
    then slowly ramp back the magnetic field to the target magnetic field

    Args:
        magnet (_type_, optional): magnet_x,magnet_y,magnet_z. Defaults to magnet_x.
        current_field (int, optional): _description_. Defaults to 0 T.
        target_field (_type_, optional): _description_. Defaults to -10e-3 T.
        step_size (_type_, optional): _description_. Defaults to 1e-3 T.
        delay (float, optional): _description_. Defaults to 2 sec.
    
    """
    
    from tqdm.notebook import tqdm
    import math
    # Calculate total steps for tqdm progress bar
    total_steps = math.ceil(abs((target_field - current_field) / step_size)) * 2
    progress_bar = tqdm(total=total_steps)

    # Determine the intermediate target field
    intermediate_target_field = target_field + 5e-3 if target_field > current_field else target_field - 5e-3
    # Slowly ramp the magnetic field to the intermediate target field
    while abs(intermediate_target_field - current_field) > step_size:
        if intermediate_target_field > current_field:
            current_field += step_size
            magnet.field(current_field)
        else:
            current_field -= step_size
            magnet.field(current_field)
        time.sleep(delay)
        progress_bar.update(1)  # Update progress bar
    # Slowly ramp back the magnetic field to the target_field
    while abs(target_field - current_field) > step_size:
        if target_field > current_field:
            current_field += step_size
            magnet.field(current_field)
        else:
            current_field -= step_size
            magnet.field(current_field)
        time.sleep(delay)
        progress_bar.update(1)  # Update progress bar
    magnet.field(target_field)
    progress_bar.n = total_steps  # Set progress bar to 100%
    progress_bar.refresh()  # Refresh the progress bar to show the update
    progress_bar.close()  # Close progress bar when done

In [ ]:
for i in range(6):
    magnet_x.field((-10-i)*1e-3)
    sleep(4)
sleep(10)

for i in range(16):
    magnet_x.field((15+i)*1e-3)
    sleep(4)
sleep(10)

In [ ]:
for i in range(25):
    print(-24+i)

In [ ]:
magnet_x.field(-19e-3)
sleep(10)
#magnet_x.field(e-3)
#sleep(10)
#magnet_x.field(12.8e-3)
#sleep(10)

## IV vs By

In [ ]:
get_Mag()

In [ ]:
magnet_y.field(0e-3)

In [ ]:
for i in tqdm(range(21), desc="2nd loop"):
    
    value = (-10 + i*0.5) * 1e-3 
    value = float(f"{value:.10f}")
    magnet_y.field(value)
    get_Mag()
    time.sleep(15)
    clear_output(True)

In [ ]:

# First loop
for i in tqdm(range(11), desc="First loop"):
    
    value = (5 +i) * 1e-3
    value = float(f"{value:.10f}")
    magnet_y.field(value)
    get_Mag()
    time.sleep(15)
    clear_output(True)

# Pause for 30 seconds
time.sleep(60)

# Second loop
for i in tqdm(range(11), desc="2nd loop"):
    
    value = (-10 + i) * 1e-3 
    value = float(f"{value:.10f}")
    magnet_y.field(value)
    get_Mag()
    time.sleep(15)
    clear_output(True)
# Pause for 180 seconds
time.sleep(300)

In [ ]:
magnet_y.field()

In [ ]:
Mag =5e-3
I_range=0.6e-6
K2440.output(1)
sleep(2)
magnet_y.field(Mag)
sleep(10)
appl_current(-I_range)
sleep(3)
direction = 'backward' if Mag > 0 else 'forward'


# station.snapshot(update=True);
exp = qc.new_experiment('IV vs By', sample_name=f'103-03_J2_27-26-30-31M(32G-33G-28G-29G)_1M_By_{direction}_{-I_range}')

dataid = do2d(magnet_y.field, Mag, -Mag, 201 , 0.5, appl_current, -I_range ,I_range,  201, 0.0001, meas_voltage_K2)

appl_current(0)
K2440.output(0)
# magnet_y.field(0)

plot_by_id(dataid)
autosave_plot_by_id(dataid, save_path)

In [ ]:
Mag =-5e-3
I_range=5e-6
K2440.output(1)
sleep(2)
magnet_y.field(Mag)
sleep(10)
appl_current(-I_range)
sleep(3)
direction = 'backward' if Mag > 0 else 'forward'


# station.snapshot(update=True);
exp = qc.new_experiment('IV vs By', sample_name=f'103-03_J2_27-26-30-31M(32G-33G-28G-29G)_1M_By_{direction}_{-I_range}')

dataid = do2d(magnet_y.field, Mag, -Mag, 201 , 0.5, appl_current, -I_range ,I_range,  201, 0.0001, meas_voltage_K2)

appl_current(0)
K2440.output(0)
# magnet_y.field(0)

plot_by_id(dataid)
autosave_plot_by_id(dataid, save_path)

In [ ]:
32-

In [ ]:

# First loop
for i in tqdm(range(6), desc="First loop"):
    Mag_x = magnet_y.field()
    value = (1.5 +i) * 1e-3
    value = float(f"{value:.10f}")
    magnet_y.field(value)
    print(f"magnet_y is {Mag_x:.8f} T")
    time.sleep(15)
    clear_output(True)

# Pause for 30 seconds
time.sleep(60)

# Second loop
for i in tqdm(range(6), desc="2nd loop"):
    Mag_x = magnet_y.field()
    value = (6.5 - i) * 1e-3 
    value = float(f"{value:.10f}")
    magnet_y.field(value)
    print(f"magnet_y is {Mag_x:.8f} T")
    time.sleep(15)
    clear_output(True)

# Pause for 180 seconds
time.sleep(300)

In [ ]:
Mag = 1.5e-3
I_range=2.5e-6
K2440.output(1)
sleep(2)
magnet_y.field(Mag)
sleep(10)
appl_current(-I_range)
sleep(3)
direction = 'backward' if Mag > 0 else 'forward'


# station.snapshot(update=True);
exp = qc.new_experiment('IV vs By', sample_name=f'103-06_J2_12-13-18-19_1M_By_{direction}_Bx@3mT')

dataid = do2d(magnet_y.field, Mag, -Mag, 1001 , 0.4, appl_current, -I_range ,I_range, 151, 0.01, meas_voltage_K2)

appl_current(0)
K2440.output(0)
# magnet_y.field(0)

plot_by_id(dataid)
autosave_plot_by_id(dataid, save_path)

In [ ]:
Mag()

In [ ]:
Mag = 1.5e-3
I_range=1e-6
K2440.output(1)
sleep(2)
magnet_y.field(Mag)
sleep(10)
appl_current(-I_range)
sleep(3)
direction = 'backward' if Mag > 0 else 'forward'


# station.snapshot(update=True);
exp = qc.new_experiment('IV vs By', sample_name=f'103-01_J1_21-20-23-22_1M_By_{direction}_Bx@0mT')

dataid = do2d(magnet_y.field, Mag, -Mag, 151 , 0.9, appl_current, -I_range ,I_range, 201, 0.01, meas_voltage_K2)

appl_current(0)
K2440.output(0)
# magnet_y.field(0)

plot_by_id(dataid)
autosave_plot_by_id(dataid, save_path)

In [ ]:
from tqdm import tqdm
import time

# First loop
for i in tqdm(range(6), desc="First loop"):
    value = (1.5 + i) * 1e-3 
    magnet_y.field(value)
    print(f"magnet_y is {value:.8f} T")
    time.sleep(10)

# Pause for 30 seconds
time.sleep(30)

# Second loop
for i in tqdm(range(8), desc="First loop"):
    value = (7 - i) * 1e-3 
    magnet_y.field(value)
    print(f"magnet_y is {value:.8f} T")
    time.sleep(10)

# Pause for 180 seconds
time.sleep(300)


In [ ]:
def combined_loop(start=0, end=5 ):
  for i in tqdm(range((end-start)+6), desc="First loop"):
    value = start + i
    print(f"{value* 1e-3:.5f}")

  for i in tqdm(range(6), desc="Second loop"):
    value = end+5 - i
    print(f"{value* 1e-3:.5f}")

combined_loop()


In [ ]:
Mag = 3e-3
I_range=1e-6
K2440.output(1)
sleep(2)
magnet_y.field(Mag)
sleep(10)
appl_current(-I_range)
sleep(3)
direction = 'backward' if Mag > 0 else 'forward'


# station.snapshot(update=True);
exp = qc.new_experiment('IV vs By', sample_name=f'103-04_J2_04-05-06-07_1M_By_{direction}_Bx@0mT')

dataid = do2d(magnet_y.field, Mag, -Mag, 101 , 1.5, appl_current, -I_range ,I_range, 101, 0.01, meas_voltage_K2)

appl_current(0)
K2440.output(0)
# magnet_y.field(0)

plot_by_id(dataid)
autosave_plot_by_id(dataid, save_path)

In [ ]:
from tqdm import tqdm
import time

# First loop
for i in tqdm(range(6), desc="First loop"):
    magnet_y.field((-3 - i) * 1e-3)
    time.sleep(15)

# Pause for 30 seconds
time.sleep(30)

# Second loop
for i in tqdm(range(7), desc="Second loop"):
    magnet_y.field((-8 + i) * 1e-3)
    time.sleep(15)

# Pause for 180 seconds
time.sleep(300)

In [ ]:
magnet_y.field(-1.5e-3)
print("By at -1.5 mT")

In [ ]:
Mag = -3e-3
I_range=1e-6
K2440.output(1)
sleep(2)
magnet_y.field(Mag)
sleep(10)
appl_current(-I_range)
sleep(3)
direction = 'backward' if Mag > 0 else 'forward'


# station.snapshot(update=True);
exp = qc.new_experiment('IV vs By', sample_name=f'103-04_J2_04-05-06-07_1M_By_{direction}_Bx@0mT')

dataid = do2d(magnet_y.field, Mag, -Mag, 501 , 1, appl_current, -I_range ,I_range, 251, 0.01, meas_voltage_K2)

appl_current(0)
K2440.output(0)
# magnet_y.field(0)

plot_by_id(dataid)
autosave_plot_by_id(dataid, save_path)

In [ ]:
from tqdm import tqdm
import time

# First loop
for i in tqdm(range(6), desc="First loop"):
    magnet_y.field((3 + i) * 1e-3)
    time.sleep(10)

# Pause for 30 seconds
time.sleep(30)

# Second loop
for i in tqdm(range(6), desc="Second loop"):
    magnet_y.field((8 - i) * 1e-3)
    time.sleep(10)

# Pause for 180 seconds
time.sleep(300)

In [ ]:
Mag = 3e-3
I_range=1e-6
K2440.output(1)
sleep(2)
magnet_y.field(Mag)
sleep(10)
appl_current(-I_range)
sleep(3)
direction = 'backward' if Mag > 0 else 'forward'


# station.snapshot(update=True);
exp = qc.new_experiment('IV vs By', sample_name=f'103-04_J2_04-05-06-07_1M_By_{direction}_Bx@0mT')

dataid = do2d(magnet_y.field, Mag, -Mag, 501 , 1, appl_current, -I_range ,I_range, 251, 0.01, meas_voltage_K2)

appl_current(0)
K2440.output(0)
# magnet_y.field(0)

plot_by_id(dataid)
autosave_plot_by_id(dataid, save_path)

In [ ]:
magnet_y.field(5e-3)

In [ ]:
for i in range(9):
    magnet_y.field((0+i)*1e-3)
    sleep(4)
sleep(10)

for i in range(6):
    magnet_y.field((8-i)*1e-3)
    sleep(4)

sleep(60)
# magnet_y.field(21.5e-3)
sleep(60)
#magnet_y.field(0.48e-3)
#sleep(5)

In [ ]:
for i in range(4):
    print(8-i)

In [ ]:
K2440.output(1)
sleep(2)
Mag = -40e-3
I_range=2e-8

magnet_y.field(Mag)
sleep(10)
appl_current(-I_range)
sleep(3)
direction = 'backward' if Mag > 0 else 'forward'


# station.snapshot(update=True);

exp = qc.new_experiment('IV vs By', sample_name=f'094-20IV_By_16-17-19-18_10M_{direction}_Bx@-18.5mT')

dataid = do2d(magnet_y.field, Mag, -Mag, 601 , 2, appl_current, -I_range ,I_range, 201, 0.1, meas_voltage_K2)


appl_current(0)
K2440.output(0)
# magnet_y.field(0)

plot_by_id(dataid)
autosave_plot_by_id(dataid, save_path)

In [ ]:
for i in range(24):
    print(45-i)

## IV vs Bz

In [ ]:
K2440.output(1)
sleep(2)
magnet_z.field(-80e-3)
sleep(60)
appl_current(-2e-7)
sleep(5)

# station.snapshot(update=True);

exp = qc.new_experiment('IV vs Bz', sample_name='IV_Bz_33_32_30_31_10M_forward')

dataid = do2d(magnet_z.field,-80e-3,80e-3, 71 , 20, appl_current, -2e-7 ,2e-7, 101, 0.05, meas_voltage_K2)

appl_current(0)

K2440.output(0)
# magnet_y.field(0)

plot_by_id(dataid)

In [ ]:
#magnet_z.field(20e-3)
#sleep(5)
magnet_z.field(-6.1e-3)
sleep(5)

## IV w/ Bx

In [ ]:
K2440.output(1)
sleep(2)
magnet_x.field(0.23e-3)
appl_current(-0.5e-6)
sleep(10)

# station.snapshot(update=True);
exp = qc.new_experiment('I vs V', sample_name='IV_w/_Bx')

#do1d(param_set, start, stop, num_points, delay, *param_meas)
dataid = do1d(appl_current, -0.5e-6 ,0.5e-6, 101, 0.05, meas_voltage_K2)
# dataid = do1d(appl_current, 6e-6 , -6e-6, 251, 0.01, meas_voltage_K1)
# ivvi.set_dacs_zero()

appl_current(0)
plot_by_id(dataid)
# print(get_R_from_IV(dataid))
#print(get_R_from_IV_2(dataid))

K2440.output(0)

## IV w/ By

In [ ]:
K2440.output(1)
DMM6500.averaging_count(30)
#DMM6500.averaging_enabled(False)

sleep(2)
magnet_y.field(0e-3)
appl_current(-2e-7)
sleep(20)

# station.snapshot(update=True);
exp = qc.new_experiment('I vs V', sample_name='IV_w/_By@0mT_forward_Avg30')

#do1d(param_set, start, stop, num_points, delay, *param_meas)
dataid = do1d(appl_current, -2e-7 ,2e-7, 201, 0.05, meas_voltage_K2)
# dataid = do1d(appl_current, 6e-6 , -6e-6, 251, 0.01, meas_voltage_K1)
# ivvi.set_dacs_zero()

appl_current(0)
plot_by_id(dataid)
# print(get_R_from_IV(dataid))
#print(get_R_from_IV_2(dataid))

K2440.output(0)

## MR

In [ ]:
K2440.output(1)
sleep(2)
magnet_y.field(-0.1)
sleep(5)
#K2440.output(1)
appl_current(1e-7)
sleep(5)

# station.snapshot(update=True);

exp = qc.new_experiment('I vs By_MR', sample_name='072_14IVf_25_24_29_29_By_+-0.1T_MR_for')

dataid = do1d(magnet_y.field, -0.1, 0.1, 71 , 1, meas_voltage_K2)

appl_current(0)
K2440.output(0)
#K2440.output(0)
magnet_y.field(0)

plot_by_id(dataid)

## IV w/Bz

In [ ]:
K2440.output(1)
DMM6500.averaging_count(10)
#DMM6500.averaging_enabled(False)

sleep(2)
magnet_z.field(0e-3)
appl_current(-2e-7)
sleep(30)

# station.snapshot(update=True);
exp = qc.new_experiment('I vs V', sample_name='IV_w/_f_Bz@0mT_forward_Avg10')

#do1d(param_set, start, stop, num_points, delay, *param_meas)
dataid = do1d(appl_current, -2e-7 ,2e-7, 201, 0.05, meas_voltage_K2)
# dataid = do1d(appl_current, 6e-6 , -6e-6, 251, 0.01, meas_voltage_K1)
# ivvi.set_dacs_zero()

appl_current(0)
plot_by_id(dataid)
# print(get_R_from_IV(dataid))
#print(get_R_from_IV_2(dataid))

K2440.output(0)

# Zero everything

In [ ]:
appl_current(0)
appl_TG(0)
magnet_y.field(0)
sleep(5)
magnet_x.field(0)
sleep(5)
magnet_z.field(0)
sleep(5)
#appl_voltage_AC(0.004)
#SGS.off()
K2440.output(0)

In [ ]:
appl_current(0)

# Gate simple sweeps

## 加TG (2D)

In [ ]:
appl_current(0)
sleep(1)
appl_TG(0)
sleep(5)

In [ ]:
magnet_x.field(-0.044e-3)
sleep(10)
magnet_y.field(-6.016e-3)
sleep(10)

In [ ]:
K2440.output(1)
K2440.output(1)
appl_TG(-30)
sleep(5)
appl_current(-1.5e-8)
sleep(1)

exp = qc.new_experiment('I_vs_Vgate_up', sample_name='33_32_30_31_1M_TG21')
dataid = do2d(appl_TG, -30, 30, 301, 0.5, appl_current, -1.5e-8, 1.5e-8, 101, 0.1, meas_voltage_K2)

appl_TG(0)
sleep(5)
appl_current(0e-7)
sleep(5)

K2440.output(0)
K2440.output(0)

plot_by_id(dataid)

# RF

## I vs RF_f

### sensitive check

In [ ]:
SGS.power()
SGS.frequency()

In [ ]:
SGS.on()

In [ ]:
SGS.off()

In [ ]:
magnet_x.field(0.4e-3)
sleep(10)
magnet_y.field(2.53e-3)
sleep(10)

In [ ]:
DMM6500.averaging_count(5)
DMM6500.averaging_enabled(False)

### I vs RF_f (1D)

In [ ]:
SGS.off()
exp = qc.new_experiment('I-RF_f', sample_name='103-06_J2_12-13-14-15_1M_@0mT')
SGS.power(0)
SGS.frequency(0.1e9)
SGS.on()
sleep(1)
DMM6500.averaging_count(5)
DMM6500.averaging_enabled(False)
K2440.output(1)

appl_current(0.4e-6)
#appl_TG(10)
# magnet_x.field(0.4e-3)
# magnet_y.field(2.53e-3)

dataid = do1d(SGS.frequency, 11.4e9, 12.2e9, 201, 0.1, meas_voltage_K2)
appl_current(0)
SGS.off()
K2440.output(0)
plot_by_id(dataid)


### I vs RF_f (2D)

In [ ]:
SGS.off()

In [ ]:
SGS.off()
SGS.power(0)
SGS.frequency(0.1e9)
SGS.on()
DMM6500.averaging_count(1)
DMM6500.averaging_enabled(False)
K2440.output(1)

I_range = 8e-6
appl_current(-I_range)
#appl_TG(-10)
# magnet_x.field(0.4e-3)
# magnet_y.field(2.53e-3)
# sleep(1)

# station.snapshot(update=True);
exp = qc.new_experiment('IV-RF_f', sample_name=f'103-03_J2_27-26-30-31M(32G-33G-28G-29G)_1M_RF@0dBm_By@0.1mT')

#do1d(param_set, start, stop, num_points, delay, *param_meas)
dataid = do2d(SGS.frequency, 1e9, 20e9, 191, 0.0001, appl_current,-I_range ,I_range, 201, 0.0001, meas_voltage_K2)
# dataid = do1d(appl_current, 6e-6 , -6e-6, 251, 0.01, meas_voltage_K1)
# ivvi.set_dacs_zero()
K2440.output(0)
SGS.off()
appl_current(0)
plot_by_id(dataid)
save_path = r"D:\data\Albert\103\Plot\104_{}.png".format(dataid)  # Path to save the plot image
autosave_plot_by_id(dataid, save_path)

In [ ]:
SGS.off()
SGS.power(20)
SGS.frequency(0.1e9)
SGS.on()
DMM6500.averaging_count(1)
DMM6500.averaging_enabled(False)
K2440.output(1)

I_range = 6e-6
appl_current(-I_range)
#appl_TG(-10)
# magnet_x.field(0.4e-3)
# magnet_y.field(2.53e-3)
# sleep(1)

# station.snapshot(update=True);
exp = qc.new_experiment('IV-RF_f', sample_name=f'103-03_J2_27-26-30-31M(32G-33G-28G-29G)_1M_RF@-10dBm_By@0.1mT')

#do1d(param_set, start, stop, num_points, delay, *param_meas)
dataid = do2d(SGS.frequency, 1e9, 20e9, 191, 0.0001, appl_current,-I_range ,I_range, 201, 0.0001, meas_voltage_K2)
# dataid = do1d(appl_current, 6e-6 , -6e-6, 251, 0.01, meas_voltage_K1)
# ivvi.set_dacs_zero()
K2440.output(0)
SGS.off()
appl_current(0)
plot_by_id(dataid)
save_path = r"D:\data\Albert\103\Plot\104_{}.png".format(dataid)  # Path to save the plot image
autosave_plot_by_id(dataid, save_path)

In [ ]:
SGS.off()
SGS.power(20)
SGS.frequency(5.9e9)
SGS.on()
DMM6500.averaging_count(1)
DMM6500.averaging_enabled(False)
K2440.output(1)

I_range = 6e-6
appl_current(-I_range)
#appl_TG(-10)
# magnet_x.field(0.4e-3)
# magnet_y.field(2.53e-3)
# sleep(1)

# station.snapshot(update=True);
exp = qc.new_experiment('IV-RF_f', sample_name=f'103-03_J2_27-26-30-31M(32G-33G-28G-29G)_1M_RF@20dBm_By@0.1mT')

#do1d(param_set, start, stop, num_points, delay, *param_meas)
dataid = do2d(SGS.frequency, 5.9e9, 6.0e9, 251, 0.0001, appl_current,-I_range ,I_range, 501, 0.0001, meas_voltage_K2)
# dataid = do1d(appl_current, 6e-6 , -6e-6, 251, 0.01, meas_voltage_K1)
# ivvi.set_dacs_zero()
K2440.output(0)
SGS.off()
appl_current(0)
plot_by_id(dataid)
save_path = r"D:\data\Albert\103\Plot\104_{}.png".format(dataid)  # Path to save the plot image
autosave_plot_by_id(dataid, save_path)


In [ ]:
SGS.off()
SGS.power(20)
SGS.frequency(5.535e9)
SGS.on()
DMM6500.averaging_count(1)
DMM6500.averaging_enabled(False)
K2440.output(1)

I_range = 6e-6
appl_current(-I_range)
#appl_TG(-10)
# magnet_x.field(0.4e-3)
# magnet_y.field(2.53e-3)
# sleep(1)

# station.snapshot(update=True);
exp = qc.new_experiment('IV-RF_f', sample_name=f'103-03_J2_27-26-30-31M(32G-33G-28G-29G)_1M_RF@20dBm_By@0.1mT')

#do1d(param_set, start, stop, num_points, delay, *param_meas)
dataid = do2d(SGS.frequency, 5.535e9, 5.585e9, 251, 0.0001, appl_current,-I_range ,I_range, 501, 0.0001, meas_voltage_K2)
# dataid = do1d(appl_current, 6e-6 , -6e-6, 251, 0.01, meas_voltage_K1)
# ivvi.set_dacs_zero()
K2440.output(0)
SGS.off()
appl_current(0)
plot_by_id(dataid)
save_path = r"D:\data\Albert\103\Plot\104_{}.png".format(dataid)  # Path to save the plot image
autosave_plot_by_id(dataid, save_path)


In [ ]:
SGS.off()
SGS.power(20)
SGS.frequency(0.1e9)
SGS.on()
DMM6500.averaging_count(1)
DMM6500.averaging_enabled(False)
K2440.output(1)

I_range = 6e-6
appl_current(-I_range)
SGS_power = SGS.power()
#appl_TG(-10)
# magnet_x.field(0.4e-3)
# magnet_y.field(2.53e-3)
# sleep(1)

# station.snapshot(update=True);
exp = qc.new_experiment('IV-RF_f', sample_name=f'103-03_J2_27-26-30-31M(32G-33G-28G-29G)_1M_RF@{SGS_power}dBm_By@0.1mT')

#do1d(param_set, start, stop, num_points, delay, *param_meas)
dataid = do2d(SGS.frequency, 3.9e9, 6.9e9, 301, 0.0001, appl_current,-I_range ,I_range, 251, 0.0001, meas_voltage_K2)
# dataid = do1d(appl_current, 6e-6 , -6e-6, 251, 0.01, meas_voltage_K1)
# ivvi.set_dacs_zero()
K2440.output(0)
SGS.off()
appl_current(0)
plot_by_id(dataid)
save_path = r"D:\data\Albert\103\Plot\104_{}.png".format(dataid)  # Path to save the plot image
autosave_plot_by_id(dataid, save_path)

In [ ]:
appl_current(0)
#appl_TG(0)
magnet_y.field(0)
sleep(2)
magnet_x.field(0)
sleep(2)
#appl_voltage_AC(0.004)
SGS.off()

## IV vs RF_f

### Shapiro steps 

In [ ]:
SGS.off()
SGS.power(-10)
SGS.frequency(11.725e9)
SGS.on()
DMM6500.averaging_count(5)
DMM6500.averaging_enabled(False)
K2440.output(1)

appl_current(-2e-7)
#appl_TG(-10)
#magnet_x.field(0.23e-3)
#magnet_y.field(1.41e-3)
# sleep(1)

# station.snapshot(update=True);
exp = qc.new_experiment('IV-RF_f', sample_name=f'103-03_J2_27-26-30-31M(32G-33G-28G-29G)_1M_By@0.1mT{-I_range}')

#do1d(param_set, start, stop, num_points, delay, *param_meas)""
dataid = do1d(appl_current,-2e-7 ,2e-7, 101, 0.05,  meas_voltage_K2)
# dataid = do1d(appl_current, 6e-6 , -6e-6, 251, 0.01, meas_voltage_K1)
# ivvi.set_dacs_zero()
K2440.output(0)
SGS.off()
appl_current(0)
plot_by_id(dataid)
# print(get_R_from_IV(dataid))
#print(get_R_from_IV_2(dataid))

In [ ]:
SGS.off()
SGS.power(0)
SGS.frequency(3.87e9)
SGS.on()
DMM6500.averaging_count(5)
DMM6500.averaging_enabled(False)
K2440.output(1)

appl_current(-1.2e-6)
#appl_TG(-10)
magnet_y.field(-10e-3)

#magnet_y.field(1.41e-3)
# sleep(1)

# station.snapshot(update=True);
exp = qc.new_experiment('Bx vs RF_f', sample_name='By vs RF_f_J2_3.87_-10dB')

#do1d(param_set, start, stop, num_points, delay, *param_meas)""
dataid = do2d(magnet_y.field, -10e-3, 10e-3,701, 1, appl_current,-1.2e-6 ,1.2e-6, 101, 0.05,  meas_voltage_K2)
# dataid = do1d(appl_current, 6e-6 , -6e-6, 251, 0.01, meas_voltage_K1)
# ivvi.set_dacs_zero()
K2440.output(0)
SGS.off()
appl_current(0)
plot_by_id(dataid)
# print(get_R_from_IV(dataid))
#print(get_R_from_IV_2(dataid))

In [ ]:
appl_current(0)
#appl_TG(0)
magnet_y.field(0)
sleep(10)
magnet_x.field(0)
sleep(10)
#appl_voltage_AC(0.004)
SGS.off()

## IV vs Power Dependence

In [ ]:
SGS.off()

In [ ]:
SGS.power(-40)

In [ ]:
SGS.off()

In [ ]:
SGS.off()
SGS.power(-20)
SGS.frequency(5.9534e9)
SGS_frequency = SGS.frequency()
SGS.on()
DMM6500.averaging_count(3)
DMM6500.averaging_enabled(False)
K2440.output(1)

#magnet_x.field(15e-3)
#sleep(5)
I_range = 6e-6
appl_current(-I_range)
#appl_TG(-30)
sleep(2)

# station.snapshot(update=True);
exp = qc.new_experiment('IV-RF_p', sample_name=f'103-03_J2_27-26-30-31M(32G-33G-28G-29G)_1M_RF@{SGS_frequency*1e-9:.4f}GHz_By@0.1mT')

#do1d(param_set, start, stop, num_points, delay, *param_meas)

dataid = do2d(SGS.power,10, 15, 51, 0.1, appl_current,-I_range ,I_range, 301, 0.00001, meas_voltage_K2)
# dataid = do1d(appl_current, 6e-6 , -6e-6, 251, 0.01, meas_voltage_K1)
# ivvi.set_dacs_zero()

SGS.off()
appl_current(0)
K2440.output()
plot_by_id(dataid)

# print(get_R_from_IV(dataid))
print(get_R_from_IV_2(dataid))

In [ ]:
SGS.off()
SGS.power(-25)
SGS.frequency(5.226e9)
SGS.on()
DMM6500.averaging_count(5)
DMM6500.averaging_enabled(True)
K2440.output(1)

magnet_y.field(-8e-3)
sleep(60)
appl_current(-2e-7)
#appl_TG(-30)
sleep(2)

# station.snapshot(update=True);
exp = qc.new_experiment('IV vs power dependence', sample_name='IV vs PD_5.226_By:-8mT')

#do1d(param_set, start, stop, num_points, delay, *param_meas)

dataid = do2d(SGS.power,-25, 10, 101, 20, appl_current,-2e-7 ,2e-7, 101, 0.05, meas_voltage_K2)
# dataid = do1d(appl_current, 6e-6 , -6e-6, 251, 0.01, meas_voltage_K1)
# ivvi.set_dacs_zero()

SGS.off()
appl_current(0)
K2440.output()
plot_by_id(dataid)

# print(get_R_from_IV(dataid))
print(get_R_from_IV_2(dataid))

### PD vs frequency_IV loop (not yet)

In [ ]:
frequencies = [3.248, 3.387, 5.677]
power_levels = [-20, 0, 25]
     
SGS.off()
SGS.power(-20)
SGS.frequency(3.387e9)
SGS.on()
DMM6500.averaging_count(5)
DMM6500.averaging_enabled(True)
K2440.output(1)

appl_current(-2e-6)
#appl_TG(-10)
sleep(1)

for freq in frequencies:
    SGS.frequency(freq)
    for power_level in power_levels:
        SGS.power(power_level)

# station.snapshot(update=True);
exp = qc.new_experiment('IV vs power dependence', sample_name='IV vs PD_J2_3.387_m20')
dataid = do2d(SGS.power, 4, 8, 100, 2, appl_current, -1e-6, 1e-6, 101, 0.05, meas_voltage_K2)

 # 關閉設備，並在需要時列印或繪製結果
SGS.off()
appl_current(0)
K2440.output()
plot_by_id(dataid)
    
print(f"頻率：{freq} Hz，功率水平：{power_level} dB")
print(get_R_from_IV_2(dataid))


## IV vs Vgate

In [ ]:
appl_current(0)
appl_TG(0)
magnet_y.field(0)
sleep(10)
magnet_x.field(0)
sleep(10)
appl_voltage_AC(0.004)
SGS.off()

In [ ]:
SGS.off()
SGS.power(25)
SGS.frequency(4.83e9)
SGS.on()
DMM6500.averaging_count(5)
DMM6500.averaging_enabled(True)
appl_current(-0.4e-7)
appl_current_AC(0.1e-10)

appl_TG(-10)
sleep(1)

# station.snapshot(update=True);
exp = qc.new_experiment('IV vs Vgate', sample_name='IV vs Vg')

dataid = do2d(appl_TG, -10, 10, 751, 0.5, appl_current, -0.4e-7, 0.4e-7, 101, 0.01, meas_voltage_AC_L1, meas_voltage_AC_L1_P, meas_voltage_K2)
# dataid = do1d(appl_current, 6e-6 , -6e-6, 251, 0.01, meas_voltage_K1)
# ivvi.set_dacs_zero()
SGS.off()
appl_current(0)
plot_by_id(dataid)
# print(get_R_from_IV(dataid))
#print(get_R_from_IV_2(dataid))

In [ ]:
appl_current(0)
appl_TG(0)
magnet_y.field(0)
sleep(10)
magnet_x.field(0)
sleep(10)
appl_voltage_AC(0.004)
SGS.off()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Constants
k = 8.617333262145e-5  # Boltzmann constant in eV/K
E_a = 0.5  # Example value for the activation energy in eV
p_0 = 1.0  # Example value for p0

# Create an array of temperatures from 100 to 1000 K
temperatures = np.linspace(100, 1000, 50)

# Calculate 1/T
inverse_temperatures = 1 / temperatures

# Calculate ln(p)
ln_p = np.log(p_0) - (E_a / k) * inverse_temperatures

# Plotting
plt.figure(figsize=(8, 6))
plt.plot(inverse_temperatures, ln_p, marker='o', linestyle='-')
plt.xlabel('1/T (1/K)')
plt.ylabel('ln(p)')
plt.title('ln(p) vs 1/T for Hole Concentration')
plt.grid(True)
plt.show()
